In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_dataset = datagen.flow_from_directory("/kaggle/input/surface-crack-detection/",
                                         target_size=(150,150),
                                         batch_size=64,
                                         shuffle=True,
                                         subset="training",
                                         class_mode="binary")

In [ ]:
validation_dataset = datagen.flow_from_directory("/kaggle/input/surface-crack-detection/",
                                         target_size=(150,150),
                                         batch_size=64,
                                         shuffle=True,
                                         subset="validation",
                                         class_mode="binary")

In [ ]:
loss = keras.losses.BinaryCrossentropy()

In [ ]:
model = keras.Sequential([layers.Input(shape=(150, 150, 3)),
                         layers.Conv2D(filters=16, kernel_size=3, activation="relu"),
                         layers.MaxPooling2D(pool_size=(2,2)),
                         layers.Flatten(),
                         layers.Dense(units=32, activation="relu"),
                         layers.Dense(units=1, activation="sigmoid")], name="cnn")

In [ ]:
model.summary()

In [ ]:
epochs = 10
batch_size = 64
learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)